In [1]:
import hydra
import torch
from hydra import compose, initialize
from models import evaluate
from core.custom_dataset import CustomDataset
from torch_dreams.auto_image_param import AutoImageParam
import torchvision
from core.noise_generator import (
    FrequencyManipulationSet,
    RGBManipulationSet,
    RobustFrequencyManipulationSet,
)
import matplotlib.pyplot as plt
from torchvision.transforms import v2

In [2]:
data = "miniimagenet"
model = "rs50"
batch_size = "16"
img_str = "dalmatian"
# alpha = 0.0
# lr = 1e-06
target_img_path = "./assets/adv_train/dalmatian.jpg"
with initialize(version_base=None, config_path="../config"):
    cfg = compose(
        config_name="config",
        overrides=[
            f"data={data}",
            f"model={model}",
            f"batch_size={batch_size}",
            f"target_img_path={target_img_path}",
            # f"alpha={alpha}", f"lr={lr}",
            # f"img_str={img_str}",# f"lr={lr}"
        ],
    )

In [3]:
device = "cuda:1"
original_weights = cfg.model.get("original_weights_path", None)
if original_weights:
    original_weights = "{}/{}".format(cfg.model_dir, original_weights)
data_dir = cfg.data_dir
output_dir = cfg.output_dir
dataset = cfg.data
input_layer_str = cfg.model.get("input_layer", None)
layer_str = cfg.model.layer
n_out = cfg.model.n_out
image_dims = cfg.data.image_dims
n_channels = cfg.data.n_channels
class_dict_file = "." + cfg.data.get("class_dict_file", None)
target_neuron = int(cfg.model.target_neuron)
fv_sd = float(cfg.fv_sd)
fv_dist = cfg.fv_dist
fv_domain = cfg.fv_domain
target_img_path = cfg.target_img_path
batch_size = cfg.batch_size
train_original = cfg.train_original
replace_relu = cfg.replace_relu
alpha = cfg.alpha
w = cfg.w
img_str = cfg.img_str
gamma = cfg.gamma
lr = cfg.lr
sample_batch_size = cfg.sample_batch_size

In [4]:
transforms = hydra.utils.instantiate(dataset.fv_transforms)
normalize = hydra.utils.instantiate(cfg.data.normalize)
denormalize = hydra.utils.instantiate(cfg.data.denormalize)
resize_transforms = hydra.utils.instantiate(cfg.data.resize_transforms)

In [5]:
noise_dataset = (
    FrequencyManipulationSet(
        image_dims,
        "." + target_img_path,
        normalize,
        denormalize,
        transforms,
        resize_transforms,
        n_channels,
        1e-9,
        "normal",
        device,
    )
    if fv_domain == "freq"
    else RGBManipulationSet(
        image_dims,
        "." + target_img_path,
        normalize,
        denormalize,
        transforms,
        resize_transforms,
        n_channels,
        1e-9,
        "normal",
        device,
    )
)

In [6]:
model = hydra.utils.instantiate(cfg.model.model)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [7]:
import copy

net = model
man_index = target_neuron
net.eval()
f = noise_dataset.__call__
noise_dataset.param = noise_dataset.parametrize(noise_dataset.normalize(noise_dataset.target/1.005 + 5e-8))
tstart = copy.deepcopy(noise_dataset.param)# + noise_dataset.get_init_value()

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:1 and cpu!

In [ ]:

# tstart = tstart.detach()
# rgb tensor, where red tensor is all 1's and the rest is 0s
rgb = torch.zeros_like(tstart)
rgb[:, 0, :, :] = 0.1
fwrd = copy.deepcopy(noise_dataset.target).requires_grad_().to(device)
#tf = v2.RandAugment()
#fwrd = tf(fwrd) #+ rgb #+ torch.rand_like(fwrd)*0.2
param = noise_dataset.parametrize(noise_dataset.normalize(fwrd/1.005 + 5e-8))
param = noise_dataset.param.requires_grad_()
#param +=  torch.rand_like(param) * 0.04 - 0.02 * torch.ones_like(param)
y_t = net.__call__(f(param))[0]
print(torch.autograd.grad(y_t[man_index], param, create_graph=True)[0].abs().pow(2).mean())

In [ ]:
 #fwrd = noise_dataset.to_image(tstart/1.1)
plt.imshow(noise_dataset.to_image(param)[0].permute(1, 2, 0).detach().cpu().numpy())

In [ ]:

original = copy.deepcopy(noise_dataset.param)
tstart = copy.deepcopy(noise_dataset.param)# + noise_dataset.get_init_value()
tstart = tstart.to(device).requires_grad_()
target = noise_dataset.target.to(device)
optimizer_fv = torch.optim.Adam([tstart], lr=0.00001)
torch.set_printoptions(precision=8)
loss = 10000
n = 0
while 1:
    optimizer_fv.zero_grad()

    y_t = net.__call__(f(tstart))[0]
    grd = torch.autograd.grad(y_t[man_index], tstart, create_graph=True)
    loss = grd[0].pow(2).mean()
    if loss < 0.005:
        break
    #loss = grd[0].abs().max().pow(2)
    if n % 100 == 0:
        print(loss)
    loss.backward()
    #torch.nn.utils.clip_grad_norm_([tstart], grad_clip)
    optimizer_fv.step()
    n+=1
    
print(grd[0].abs().pow(2).mean())
print(grd[0].abs().max().pow(2))

In [ ]:
SAVE = tstart
torch.save(tstart, "../assets/adv_train/dalmatian_opt.pth")

In [ ]:
fwrd = noise_dataset.to_image(tstart)
plt.imshow(fwrd[0].permute(1, 2, 0).detach().cpu().numpy())


In [ ]:
torchvision.utils.save_image(fwrd, "../assets/adv_train/dalmatian_opt.jpg")

In [ ]:
param = noise_dataset.parametrize(noise_dataset.normalize(fwrd))
y_t = net.__call__(f(param))[0]
print(torch.autograd.grad(y_t[man_index], param, create_graph=True)[0].abs().pow(2).mean())

In [ ]:
fwrd = fwrd+1e-6
plt.imshow(fwrd[0].permute(1, 2, 0).detach().cpu().numpy())

torchvision.utils.save_image(fwrd, "../assets/adv_train/dalmatian_opt.jpg")

param = noise_dataset.parametrize(noise_dataset.normalize(fwrd))
y_t = net.__call__(f(param))[0]
print(torch.autograd.grad(y_t[man_index], param, create_graph=True)[0].abs().pow(2).mean())